In [ ]:
import pandas as pd
import numpy as np
import googlemaps
from datetime import datetime
from typing import Optional, Dict, List, Tuple
import functools
import json
import time
import random
from tqdm import tqdm

# === CONFIGURATION ===
# Load API key
try:
    with open("secret.json") as f:
        secrets = json.load(f)
    API_KEY = secrets.get("GOOGLE_API_KEY")
    if not API_KEY:
        raise ValueError("API key not found in secret.json")
except FileNotFoundError:
    raise FileNotFoundError("secret.json not found. Please add your API key.")

# Company address
company_address = "1362 Av. des Platanes, 34970 Lattes"

# === GOOGLE MAPS FUNCTIONS ===
# Cache decorator for API calls
def cache_results(func):
    """Simple memoization decorator to cache API responses"""
    cache = {}
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Skip client object for cache key
        key_args = args[1:]  # Skip client (first arg)
        cache_key = str(key_args) + str(sorted(kwargs.items()))
        if cache_key not in cache:
            cache[cache_key] = func(*args, **kwargs)
        return cache[cache_key]
    return wrapper

@cache_results
def geocode_address(client: googlemaps.Client, address: str) -> Optional[Tuple[float, float]]:
    """Convert address to coordinates using Geocoding API"""
    try:
        result = client.geocode(address)
        if result:
            location = result[0]['geometry']['location']
            return (location['lat'], location['lng'])
    except Exception as e:
        print(f"Geocoding failed for '{address}': {str(e)}")
    return None

def calculate_route(
    client: googlemaps.Client,
    origin: str,
    destination: str,
    mode: str = "driving",
    alternatives: bool = False,
    units: str = "metric"
) -> Optional[Dict]:
    """Calculate route between two locations using Routes API"""
    try:
        directions = client.directions(
            origin,
            destination,
            mode=mode,
            departure_time=datetime.now(),
            alternatives=alternatives,
            units=units
        )
        
        if not directions:
            return None
            
        primary_route = directions[0]
        leg = primary_route['legs'][0]
        
        result = {
            "origin": leg['start_address'],
            "destination": leg['end_address'],
            "distance": leg['distance']['text'],
            "distance_meters": leg['distance']['value'],
            "duration": leg['duration']['text'],
            "duration_seconds": leg['duration']['value'],
            "travel_mode": mode.upper(),
            "steps": [{
                "instruction": step['html_instructions'],
                "distance": step['distance']['text'],
                "duration": step['duration']['text']
            } for step in leg['steps']]
        }
        
        if alternatives and len(directions) > 1:
            result["alternatives"] = [{
                "summary": alt['summary'],
                "distance": alt['legs'][0]['distance']['text'],
                "duration": alt['legs'][0]['duration']['text']
            } for alt in directions[1:]]
        
        return result
        
    except Exception as e:
        print(f"Route calculation failed for {mode} mode: {str(e)}")
        return None

def get_route_between_addresses(
    client: googlemaps.Client,
    origin_address: str,
    destination_address: str,
    **kwargs
) -> Optional[Dict]:
    """Higher-level function that handles geocoding and routing"""
    origin_coords = geocode_address(client, origin_address)
    destination_coords = geocode_address(client, destination_address)
    
    if not origin_coords or not destination_coords:
        print(f"Could not geocode addresses: {origin_address} -> {destination_address}")
        return None
        
    return calculate_route(
        client,
        origin=origin_coords,
        destination=destination_coords,
        **kwargs
    )

# === DATA PROCESSING FUNCTIONS ===
def parse_duration(duration_text: str) -> str:
    """Convert Google Maps duration to hh:mm:ss format"""
    try:
        # Handle formats like "5 hours 36 mins", "33 mins", "1 day 2 hours", etc.
        parts = duration_text.split()
        hours = 0
        minutes = 0
        seconds = 0
        
        for i in range(len(parts)):
            if parts[i].isdigit():
                value = int(parts[i])
                if i + 1 < len(parts):
                    if 'hour' in parts[i+1] or 'heure' in parts[i+1]:
                        hours = value
                    elif 'min' in parts[i+1] or 'minute' in parts[i+1]:
                        minutes = value
                    elif 'day' in parts[i+1] or 'jour' in parts[i+1]:
                        hours = value * 24
                    elif 'sec' in parts[i+1] or 'seconde' in parts[i+1]:
                        seconds = value
        
        # Format as hh:mm:ss
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    except Exception as e:
        print(f"Error parsing duration '{duration_text}': {e}")
        return "00:00:00"

def parse_distance(distance_text: str) -> float:
    """Extract distance in km from Google Maps response"""
    try:
        # Handle formats like "24.6 km", "300 m", "1.2 mi"
        if 'km' in distance_text:
            return float(distance_text.split()[0].replace(',', '.'))
        elif 'm' in distance_text:
            return float(distance_text.split()[0].replace(',', '.')) / 1000
        elif 'mi' in distance_text:
            # Convert miles to km
            return float(distance_text.split()[0].replace(',', '.')) * 1.60934
        else:
            # Assume km if no unit specified
            return float(distance_text.split()[0].replace(',', '.'))
    except Exception as e:
        print(f"Error parsing distance '{distance_text}': {e}")
        return 0.0

def get_employee_commute(row: pd.Series, company_addr: str) -> Dict:
    """Calculate commute for a single employee with better error handling"""
    # Map transportation modes to Google Maps modes
    transport_mapping = {
        'Transports en commun': 'transit',
        'véhicule thermique/électrique': 'driving',
        'Marche/running': 'walking',
        'Vélo/Trottinette/Autres': 'bicycling'
    }
    
    try:
        employee_name = f"{row.get('Prénom', 'Unknown')} {row.get('Nom', 'Unknown')}"
        home_address = row.get('Adresse du domicile', '')
        
        if not home_address:
            print(f"⚠️  No address found for {employee_name}")
            return {
                'Distance_km': None,
                'Duree_hhmmss': "00:00:00",
                'Success': False
            }
        
        mode = transport_mapping.get(row.get('Moyen de déplacement', ''), 'driving')
        
        print(f"📍 Calculating route for {employee_name} ({mode})...")
        
        # Get route using your existing function
        result = get_route_between_addresses(
            client=client,
            origin_address=home_address,
            destination_address=company_addr,
            mode=mode,
            alternatives=False
        )
        
        if result:
            return {
                'Distance_km': parse_distance(result['distance']),
                'Duree_hhmmss': parse_duration(result['duration']),
                'Success': True
            }
        else:
            print(f"❌ No route found for {employee_name}")
            return {
                'Distance_km': None,
                'Duree_hhmmss': "00:00:00",
                'Success': False
            }
            
    except Exception as e:
        print(f"❌ Error processing {row.get('Nom', 'Unknown')} {row.get('Prénom', 'Unknown')}: {str(e)}")
        return {
            'Distance_km': None,
            'Duree_hhmmss': "00:00:00",
            'Success': False
        }

def calculate_all_commutes(df: pd.DataFrame, company_addr: str) -> pd.DataFrame:
    """Calculate commutes for all employees and return enhanced DataFrame"""
    
    # Initialize Google Maps client
    global client
    client = googlemaps.Client(key=API_KEY)
    
    results = []
    successful_calls = 0
    failed_calls = 0
    
    print(f"🚀 Starting commute calculation for {len(df)} employees...")
    print(f"🏢 Company address: {company_addr}")
    print("-" * 60)
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Calculating commutes"):
        try:
            commute_data = get_employee_commute(row, company_addr)
            
            if commute_data['Success']:
                successful_calls += 1
            else:
                failed_calls += 1
                
            results.append(commute_data)
            
            # Rate limiting to avoid API quota issues
            time.sleep(random.uniform(1.0, 2.0))
            
        except Exception as e:
            print(f"💥 Critical error processing row {idx}: {e}")
            results.append({
                'Distance_km': None,
                'Duree_hhmmss': "00:00:00",
                'Success': False
            })
            failed_calls += 1
    
    # Add results to DataFrame
    df_result = df.copy()
    df_result['Distance_km'] = [r['Distance_km'] for r in results]
    df_result['Duree_hhmmss'] = [r['Duree_hhmmss'] for r in results]
    df_result['Commute_Success'] = [r['Success'] for r in results]
    
    print("-" * 60)
    print(f"✅ Processing complete!")
    print(f"📊 Success: {successful_calls}, Failed: {failed_calls}")
    print(f"📈 Success rate: {(successful_calls/len(df))*100:.1f}%")
    
    return df_result

# === MAIN EXECUTION ===
def main():
    # Load data
    print("📂 Loading data files...")
    rh_path = "../data/DonneesRH.xlsx"
    sport_path = "../data/DonneesSportive.xlsx"
    
    df_rh = pd.read_excel(rh_path)
    df_sport = pd.read_excel(sport_path)
    
    print(f"👥 RH data: {len(df_rh)} employees")
    print(f"🏃 Sport data: {len(df_sport)} employees")
    
    # Basic data validation
    print("\n🔍 Data validation...")
    print("Null values in RH data:")
    print(df_rh.isnull().sum())
    print("\nTransport modes available:")
    print(df_rh['Moyen de déplacement'].value_counts())
    
    # Calculate commutes
    df_with_commutes = calculate_all_commutes(df_rh, company_address)
    
    # Export results
    output_file = "employee_commutes.csv"
    print(f"\n💾 Saving results to {output_file}...")
    
    # Export all original columns plus the two new ones
    df_with_commutes.to_csv(output_file, index=False)
    
    # Display summary
    print("\n📋 Results summary:")
    successful_commutes = df_with_commutes[df_with_commutes['Commute_Success'] == True]
    print(f"Employees with successful commute calculation: {len(successful_commutes)}")
    
    if len(successful_commutes) > 0:
        print(f"Average distance: {successful_commutes['Distance_km'].mean():.1f} km")
        print(f"Average duration: {successful_commutes['Duree_hhmmss'].mode().iloc[0] if not successful_commutes['Duree_hhmmss'].empty else 'N/A'}")
        
        print("\n👤 First 5 employees with commute info:")
        display_cols = ['Nom', 'Prénom', 'Adresse du domicile', 'Moyen de déplacement', 'Distance_km', 'Duree_hhmmss']
        print(successful_commutes[display_cols].head().to_string(index=False))
    
    print(f"\n🎉 All done! File saved as: {output_file}")

# === TEST FUNCTION ===
def test_single_employee():
    """Test the function with a single employee"""
    print("🧪 Testing with single employee...")
    
    # Initialize client
    global client
    client = googlemaps.Client(key=API_KEY)
    
    # Test address
    test_address = "128 Rue du Port, 34000 Frontignan"
    
    # Test different modes
    modes = ['driving', 'walking', 'bicycling', 'transit']
    
    for mode in modes:
        print(f"\nTesting {mode} mode...")
        result = get_route_between_addresses(
            client=client,
            origin_address=test_address,
            destination_address=company_address,
            mode=mode
        )
        
        if result:
            distance_km = parse_distance(result['distance'])
            duration_hhmmss = parse_duration(result['duration'])
            print(f"✅ {mode}: {distance_km} km, {duration_hhmmss}")
        else:
            print(f"❌ {mode}: No route found")

# Run the script
if __name__ == "__main__":
    # test_single_employee()
    main()

📂 Loading data files...
👥 RH data: 161 employees
🏃 Sport data: 161 employees

🔍 Data validation...
Null values in RH data:
ID salarié               0
Nom                      0
Prénom                   0
Date de naissance        0
BU                       0
Date d'embauche          0
Salaire brut             0
Type de contrat          0
Nombre de jours de CP    0
Adresse du domicile      0
Moyen de déplacement     0
dtype: int64

Transport modes available:
Moyen de déplacement
véhicule thermique/électrique    73
Vélo/Trottinette/Autres          54
Transports en commun             20
Marche/running                   14
Name: count, dtype: int64
🚀 Starting commute calculation for 161 employees...
🏢 Company address: 1362 Av. des Platanes, 34970 Lattes
------------------------------------------------------------


Calculating commutes:   0%|          | 0/161 [00:00<?, ?it/s]

📍 Calculating route for Audrey Colin (transit)...


Calculating commutes:   1%|          | 1/161 [00:01<04:29,  1.69s/it]

📍 Calculating route for Monique Ledoux (driving)...


...

📍 Calculating route for Odette Dumas (driving)...


Calculating commutes:  99%|█████████▉| 160/161 [04:30<00:01,  1.56s/it]

📍 Calculating route for Henri Pineau (bicycling)...


Calculating commutes: 100%|██████████| 161/161 [04:31<00:00,  1.69s/it]


------------------------------------------------------------
✅ Processing complete!
📊 Success: 161, Failed: 0
📈 Success rate: 100.0%

💾 Saving results to employee_commutes.csv...

📋 Results summary:
Employees with successful commute calculation: 161
Average distance: 19.6 km
Average duration: 00:28:00

👤 First 5 employees with commute info:
      Nom   Prénom                            Adresse du domicile          Moyen de déplacement  Distance_km Duree_hhmmss
    Colin   Audrey              128 Rue du Port, 34000 Frontignan          Transports en commun         26.8     00:43:00
   Ledoux  Monique 68 Rue du Port, 34970 Saint-Clément-de-Rivière véhicule thermique/électrique         22.6     00:30:00
   Dumont Michelle                100 Av. de la Gare, 30900 Nîmes véhicule thermique/électrique         48.7     00:35:00
Toussaint   Judith                53 Av. de la Gare, 34970 Lattes                Marche/running          1.6     00:22:00
   Bailly Michelle                74 Rue des Fl

In [7]:
test_single_employee()

🧪 Testing with single employee...

Testing driving mode...
✅ driving: 26.7 km, 00:29:00

Testing walking mode...
✅ walking: 24.6 km, 05:36:00

Testing bicycling mode...
✅ bicycling: 27.9 km, 01:33:00

Testing transit mode...
✅ transit: 40.5 km, 01:12:00


In [11]:
import pandas as pd
import numpy as np
import googlemaps
from datetime import datetime, timedelta, time as dt_time
from typing import Optional, Dict, List, Tuple
import functools
import json
import time
import random
from tqdm import tqdm

# === CONFIGURATION ===
try:
    with open("secret.json") as f:
        secrets = json.load(f)
    API_KEY = secrets.get("GOOGLE_API_KEY")
    if not API_KEY:
        raise ValueError("API key not found in secret.json")
except FileNotFoundError:
    raise FileNotFoundError("secret.json not found.")

company_address = "1362 Av. des Platanes, 34970 Lattes"

# === IMPROVED ROUTE CALCULATION WITH MULTIPLE TIMES ===
def cache_results(func):
    cache = {}
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        key_args = args[1:]
        cache_key = str(key_args) + str(sorted(kwargs.items()))
        if cache_key not in cache:
            cache[cache_key] = func(*args, **kwargs)
        return cache[cache_key]
    return wrapper

@cache_results
def geocode_address(client: googlemaps.Client, address: str) -> Optional[Tuple[float, float]]:
    try:
        result = client.geocode(address)
        if result:
            location = result[0]['geometry']['location']
            return (location['lat'], location['lng'])
    except Exception as e:
        print(f"Geocoding failed for '{address}': {str(e)}")
    return None

def calculate_route_with_options(
    client: googlemaps.Client,
    origin: str,
    destination: str,
    mode: str = "driving",
    departure_time: Optional[datetime] = None,
    traffic_model: str = "best_guess"
) -> Optional[Dict]:
    """Calculate route with specific timing options"""
    try:
        api_params = {
            'origin': origin,
            'destination': destination,
            'mode': mode,
            'units': 'metric'
        }
        
        # Add departure time and traffic model if provided
        if departure_time:
            api_params['departure_time'] = departure_time
            if mode == 'driving' and traffic_model:
                api_params['traffic_model'] = traffic_model
        
        directions = client.directions(**api_params)
        
        if not directions:
            return None
            
        primary_route = directions[0]
        leg = primary_route['legs'][0]
        
        # Use traffic duration if available
        duration_data = leg.get('duration_in_traffic', leg['duration'])
        
        result = {
            "origin": leg['start_address'],
            "destination": leg['end_address'],
            "distance": leg['distance']['text'],
            "distance_meters": leg['distance']['value'],
            "duration": duration_data['text'],
            "duration_seconds": duration_data['value'],
            "travel_mode": mode.upper(),
            "has_traffic_data": 'duration_in_traffic' in leg,
            "departure_time": departure_time.isoformat() if departure_time else "optimistic"
        }
        
        return result
        
    except Exception as e:
        print(f"Route calculation failed: {str(e)}")
        return None

def get_multiple_route_times(
    client: googlemaps.Client,
    origin_address: str,
    destination_address: str,
    mode: str = "driving"
) -> Dict:
    """Get both optimistic and rush-hour travel times"""
    
    origin_coords = geocode_address(client, origin_address)
    destination_coords = geocode_address(client, destination_address)
    
    if not origin_coords or not destination_coords:
        return {"error": "Geocoding failed"}
    
    results = {}
    
    # 1. OPTIMISTIC TIME (no specific time = historical average)
    optimistic_route = calculate_route_with_options(
        client=client,
        origin=origin_coords,
        destination=destination_coords,
        mode=mode,
        departure_time=None,  # No specific time = best-case scenario
        traffic_model=None
    )
    
    if optimistic_route:
        results['optimistic'] = {
            'duration': optimistic_route['duration'],
            'duration_seconds': optimistic_route['duration_seconds'],
            'distance_km': optimistic_route['distance_meters'] / 1000
        }
    
    # 2. RUSH-HOUR TIME (tomorrow at 8:00 AM)
    tomorrow_8am = (datetime.now() + timedelta(days=1)).replace(
        hour=8, minute=0, second=0, microsecond=0
    )
    
    rush_hour_route = calculate_route_with_options(
        client=client,
        origin=origin_coords,
        destination=destination_coords,
        mode=mode,
        departure_time=tomorrow_8am,
        traffic_model="best_guess"
    )
    
    if rush_hour_route:
        results['rush_hour'] = {
            'duration': rush_hour_route['duration'],
            'duration_seconds': rush_hour_route['duration_seconds'],
            'distance_km': rush_hour_route['distance_meters'] / 1000
        }
    
    # 3. For transit: also get a midday time (better schedules)
    if mode == 'transit':
        tomorrow_12pm = (datetime.now() + timedelta(days=1)).replace(
            hour=12, minute=0, second=0, microsecond=0
        )
        
        midday_route = calculate_route_with_options(
            client=client,
            origin=origin_coords,
            destination=destination_coords,
            mode=mode,
            departure_time=tomorrow_12pm
        )
        
        if midday_route:
            results['midday'] = {
                'duration': midday_route['duration'],
                'duration_seconds': midday_route['duration_seconds'],
                'distance_km': midday_route['distance_meters'] / 1000
            }
    
    return results

# === DATA PROCESSING FUNCTIONS ===
def parse_duration_from_seconds(total_seconds: int) -> str:
    """Convert seconds to hh:mm:ss format"""
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def get_employee_commute_detailed(row: pd.Series, company_addr: str) -> Dict:
    """Get detailed commute info with multiple time options"""
    
    transport_mapping = {
        'Transports en commun': 'transit',
        'véhicule thermique/électrique': 'driving',
        'Marche/running': 'walking',
        'Vélo/Trottinette/Autres': 'bicycling'
    }
    
    try:
        employee_name = f"{row.get('Prénom', 'Unknown')} {row.get('Nom', 'Unknown')}"
        home_address = row.get('Adresse du domicile', '')
        
        if not home_address:
            return {
                'Distance_km_optimistic': None,
                'Duree_hhmmss_optimistic': "00:00:00",
                'Distance_km_rush_hour': None,
                'Duree_hhmmss_rush_hour': "00:00:00",
                'Success': False
            }
        
        mode = transport_mapping.get(row.get('Moyen de déplacement', ''), 'driving')
        
        print(f"📍 Calculating routes for {employee_name} ({mode})...")
        
        # Get multiple time estimates
        route_times = get_multiple_route_times(
            client=client,
            origin_address=home_address,
            destination_address=company_addr,
            mode=mode
        )
        
        if 'error' in route_times:
            print(f"❌ Error for {employee_name}: {route_times['error']}")
            return {
                'Distance_km_optimistic': None,
                'Duree_hhmmss_optimistic': "00:00:00",
                'Distance_km_rush_hour': None,
                'Duree_hhmmss_rush_hour': "00:00:00",
                'Success': False
            }
        
        # Extract results
        optimistic_data = route_times.get('optimistic', {})
        rush_hour_data = route_times.get('rush_hour', {})
        
        # For transit, prefer midday if available (better schedules)
        if mode == 'transit' and 'midday' in route_times:
            transit_data = route_times['midday']
            print(f"🚌 {employee_name}: Using midday transit schedule")
        else:
            transit_data = rush_hour_data
        
        result = {
            'Distance_km_optimistic': optimistic_data.get('distance_km'),
            'Duree_hhmmss_optimistic': parse_duration_from_seconds(optimistic_data.get('duration_seconds', 0)),
            'Distance_km_rush_hour': rush_hour_data.get('distance_km') if mode != 'transit' else transit_data.get('distance_km'),
            'Duree_hhmmss_rush_hour': parse_duration_from_seconds(
                rush_hour_data.get('duration_seconds', 0) if mode != 'transit' else transit_data.get('duration_seconds', 0)
            ),
            'Success': True
        }
        
        # Print comparison
        if optimistic_data and rush_hour_data:
            opt_time = optimistic_data.get('duration', 'N/A')
            rush_time = rush_hour_data.get('duration', 'N/A')
            print(f"✅ {employee_name}: Optimistic: {opt_time}, Rush hour: {rush_time}")
        
        return result
            
    except Exception as e:
        print(f"❌ Error processing {employee_name}: {str(e)}")
        return {
            'Distance_km_optimistic': None,
            'Duree_hhmmss_optimistic': "00:00:00",
            'Distance_km_rush_hour': None,
            'Duree_hhmmss_rush_hour': "00:00:00",
            'Success': False
        }

# === MAIN PROCESSING ===
def main():
    # Load data
    print("📂 Loading data files...")
    rh_path = "../data/DonneesRH.xlsx"
    df_rh = pd.read_excel(rh_path)
    
    # Initialize client
    global client
    client = googlemaps.Client(key=API_KEY)
    
    print(f"👥 Processing {len(df_rh)} employees...")
    print("🕐 Calculating both optimistic and rush-hour (8:00 AM) times")
    print("-" * 60)
    
    results = []
    for idx, row in tqdm(df_rh.iterrows(), total=len(df_rh), desc="Calculating commutes"):
        commute_data = get_employee_commute_detailed(row, company_address)
        results.append(commute_data)
        time.sleep(random.uniform(2.0, 3.0))  # Conservative rate limiting
    
    # Add results to DataFrame
    df_rh['Distance_km_optimistic'] = [r['Distance_km_optimistic'] for r in results]
    df_rh['Duree_hhmmss_optimistic'] = [r['Duree_hhmmss_optimistic'] for r in results]
    df_rh['Distance_km_rush_hour'] = [r['Distance_km_rush_hour'] for r in results]
    df_rh['Duree_hhmmss_rush_hour'] = [r['Duree_hhmmss_rush_hour'] for r in results]
    df_rh['Commute_Success'] = [r['Success'] for r in results]
    
    # Export
    output_file = "employee_commutes_detailed.csv"
    df_rh.to_csv(output_file, index=False)
    
    print(f"\n🎉 Done! File saved as: {output_file}")
    
    # Show summary
    successful = df_rh[df_rh['Commute_Success'] == True]
    print(f"📊 Successful calculations: {len(successful)}/{len(df_rh)}")
    
    if len(successful) > 0:
        print("\n📈 Time comparisons (average):")
        print(f"Optimistic: {successful['Duree_hhmmss_optimistic'].mode().iloc[0] if not successful['Duree_hhmmss_optimistic'].empty else 'N/A'}")
        print(f"Rush hour: {successful['Duree_hhmmss_rush_hour'].mode().iloc[0] if not successful['Duree_hhmmss_rush_hour'].empty else 'N/A'}")
        
        # Show specific employee example
        employee_data = df_rh[
            (df_rh['Nom'] == 'Colin') & 
            (df_rh['Prénom'] == 'Audrey')
        ]
        
        if not employee_data.empty:
            print(f"\n👤 Example - Audrey Colin:")
            print(f"Optimistic: {employee_data['Duree_hhmmss_optimistic'].iloc[0]}")
            print(f"Rush hour: {employee_data['Duree_hhmmss_rush_hour'].iloc[0]}")

# === TEST FUNCTION ===
def test_comparison():
    """Test the difference between optimistic and rush-hour times"""
    print("🧪 Testing time comparisons...")
    
    global client
    client = googlemaps.Client(key=API_KEY)
    
    test_address = "128 Rue du Port, 34000 Frontignan"
    
    # Test different modes
    for mode in ['transit', 'driving']:
        print(f"\n--- Testing {mode} mode ---")
        
        route_times = get_multiple_route_times(
            client=client,
            origin_address=test_address,
            destination_address=company_address,
            mode=mode
        )
        
        if 'error' not in route_times:
            optimistic = route_times.get('optimistic', {})
            rush_hour = route_times.get('rush_hour', {})
            
            print(f"Optimistic: {optimistic.get('duration', 'N/A')}")
            print(f"Rush hour: {rush_hour.get('duration', 'N/A')}")
            
            if optimistic and rush_hour:
                opt_sec = optimistic.get('duration_seconds', 0)
                rush_sec = rush_hour.get('duration_seconds', 0)
                if opt_sec and rush_sec:
                    difference = rush_sec - opt_sec
                    print(f"Time difference: {difference//60} minutes")

# Run the script
if __name__ == "__main__":
    # Test first to see the comparison
    test_comparison()
    
    # Then run the main processing
    # main()

🧪 Testing time comparisons...

--- Testing transit mode ---
Optimistic: 52 mins
Rush hour: 47 mins
Time difference: -5 minutes

--- Testing driving mode ---
Optimistic: 33 mins
Rush hour: 37 mins
Time difference: 4 minutes


In [12]:
import pandas as pd
import numpy as np
import googlemaps
from datetime import datetime, timedelta, time as dt_time
from typing import Optional, Dict, List, Tuple
import functools
import json
import time
import random
from tqdm import tqdm

# === CONFIGURATION ===
try:
    with open("secret.json") as f:
        secrets = json.load(f)
    API_KEY = secrets.get("GOOGLE_API_KEY")
    if not API_KEY:
        raise ValueError("API key not found in secret.json")
except FileNotFoundError:
    raise FileNotFoundError("secret.json not found.")

company_address = "1362 Av. des Platanes, 34970 Lattes"

# === UTILITY FUNCTIONS ===
def cache_results(func):
    cache = {}
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        key_args = args[1:]
        cache_key = str(key_args) + str(sorted(kwargs.items()))
        if cache_key not in cache:
            cache[cache_key] = func(*args, **kwargs)
        return cache[cache_key]
    return wrapper

@cache_results
def geocode_address(client: googlemaps.Client, address: str) -> Optional[Tuple[float, float]]:
    try:
        result = client.geocode(address)
        if result:
            location = result[0]['geometry']['location']
            return (location['lat'], location['lng'])
    except Exception as e:
        print(f"Geocoding failed for '{address}': {str(e)}")
    return None

def get_next_weekday(target_weekday=0, hour=10, minute=0):
    """Get next specific weekday at specified time"""
    today = datetime.now()
    days_ahead = (target_weekday - today.weekday() + 7) % 7
    if days_ahead == 0 and today.time() > dt_time(hour, minute):
        days_ahead = 7
    
    target_date = today + timedelta(days=days_ahead)
    return target_date.replace(hour=hour, minute=minute, second=0, microsecond=0)

def calculate_route_with_options(
    client: googlemaps.Client,
    origin: str,
    destination: str,
    mode: str = "driving",
    departure_time: Optional[datetime] = None,
    traffic_model: Optional[str] = None
) -> Optional[Dict]:
    """Calculate route with specific timing options"""
    try:
        api_params = {
            'origin': origin,
            'destination': destination,
            'mode': mode,
            'units': 'metric'
        }
        
        if departure_time:
            api_params['departure_time'] = departure_time
            if mode == 'driving' and traffic_model:
                api_params['traffic_model'] = traffic_model
        
        directions = client.directions(**api_params)
        
        if not directions:
            return None
            
        leg = directions[0]['legs'][0]
        duration_data = leg.get('duration_in_traffic', leg['duration'])
        
        return {
            "duration": duration_data['text'],
            "duration_seconds": duration_data['value'],
            "distance_meters": leg['distance']['value'],
            "has_traffic_data": 'duration_in_traffic' in leg,
            "departure_time": departure_time.isoformat() if departure_time else "current"
        }
        
    except Exception as e:
        print(f"Route calculation failed: {str(e)}")
        return None

def get_meaningful_route_times(
    client: googlemaps.Client,
    origin_address: str,
    destination_address: str,
    mode: str = "driving"
) -> Dict:
    """Get three meaningful time estimates"""
    
    origin_coords = geocode_address(client, origin_address)
    destination_coords = geocode_address(client, destination_address)
    
    if not origin_coords or not destination_coords:
        return {"error": "Geocoding failed"}
    
    results = {}
    
    # 1. TYPICAL CONDITIONS (Monday 10:00 AM - off-peak)
    monday_10am = get_next_weekday(0, 10, 0)  # Monday 10:00 AM
    
    typical_route = calculate_route_with_options(
        client=client,
        origin=origin_coords,
        destination=destination_coords,
        mode=mode,
        departure_time=monday_10am,
        traffic_model="optimistic" if mode == "driving" else None
    )
    
    if typical_route:
        results['typical'] = {
            'duration': typical_route['duration'],
            'duration_seconds': typical_route['duration_seconds'],
            'distance_km': typical_route['distance_meters'] / 1000,
            'description': 'Typical conditions (Monday 10:00 AM)'
        }
    
    # 2. RUSH HOUR (Tomorrow 8:00 AM)
    tomorrow_8am = (datetime.now() + timedelta(days=1)).replace(hour=8, minute=0, second=0)
    
    rush_hour_route = calculate_route_with_options(
        client=client,
        origin=origin_coords,
        destination=destination_coords,
        mode=mode,
        departure_time=tomorrow_8am,
        traffic_model="best_guess" if mode == "driving" else None
    )
    
    if rush_hour_route:
        results['rush_hour'] = {
            'duration': rush_hour_route['duration'],
            'duration_seconds': rush_hour_route['duration_seconds'],
            'distance_km': rush_hour_route['distance_meters'] / 1000,
            'description': 'Rush hour (8:00 AM)'
        }
    
    # current_route = calculate_route_with_options(
    #     client=client,
    #     origin=origin_coords,
    #     destination=destination_coords,
    #     mode=mode,
    #     departure_time=datetime.now()
    # )
    
    # if current_route:
    #     results['current'] = {
    #         'duration': current_route['duration'],
    #         'duration_seconds': current_route['duration_seconds'],
    #         'distance_km': current_route['distance_meters'] / 1000,
    #         'description': 'Current conditions'
    #     }
    
    return results

def parse_duration_from_seconds(total_seconds: int) -> str:
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def get_employee_commute_corrected(row: pd.Series, company_addr: str) -> Dict:
    transport_mapping = {
        'Transports en commun': 'transit',
        'véhicule thermique/électrique': 'driving',
        'Marche/running': 'walking',
        'Vélo/Trottinette/Autres': 'bicycling'
    }
    
    try:
        employee_name = f"{row.get('Prénom', 'Unknown')} {row.get('Nom', 'Unknown')}"
        home_address = row.get('Adresse du domicile', '')
        
        if not home_address:
            return {
                'Distance_km_typical': None,
                'Duree_hhmmss_typical': "00:00:00",
                'Distance_km_rush_hour': None,
                'Duree_hhmmss_rush_hour': "00:00:00",
                # 'Distance_km_current': None,
                # 'Duree_hhmmss_current': "00:00:00",
                'Success': False
            }
        
        mode = transport_mapping.get(row.get('Moyen de déplacement', ''), 'driving')
        
        print(f"📍 Calculating routes for {employee_name} ({mode})...")
        
        route_times = get_meaningful_route_times(
            client=client,
            origin_address=home_address,
            destination_address=company_addr,
            mode=mode
        )
        
        if 'error' in route_times:
            return {
                'Distance_km_typical': None,
                'Duree_hhmmss_typical': "00:00:00",
                'Distance_km_rush_hour': None,
                'Duree_hhmmss_rush_hour': "00:00:00",
                # 'Distance_km_current': None,
                # 'Duree_hhmmss_current': "00:00:00",
                'Success': False
            }
        
        # Extract results
        typical_data = route_times.get('typical', {})
        rush_hour_data = route_times.get('rush_hour', {})
        # current_data = route_times.get('current', {})
        
        result = {
            'Distance_km_typical': typical_data.get('distance_km'),
            'Duree_hhmmss_typical': parse_duration_from_seconds(typical_data.get('duration_seconds', 0)),
            'Distance_km_rush_hour': rush_hour_data.get('distance_km'),
            'Duree_hhmmss_rush_hour': parse_duration_from_seconds(rush_hour_data.get('duration_seconds', 0)),
            # 'Distance_km_current': current_data.get('distance_km'),
            # 'Duree_hhmmss_current': parse_duration_from_seconds(current_data.get('duration_seconds', 0)),
            'Success': True
        }
        
        # Print results
        if typical_data and rush_hour_data:
            print(f"✅ {employee_name}:")
            print(f"   Typical: {typical_data.get('duration')}")
            print(f"   Rush hour: {rush_hour_data.get('duration')}")
            # print(f"   Current: {current_data.get('duration')}")
        
        return result
            
    except Exception as e:
        print(f"❌ Error processing {employee_name}: {str(e)}")
        return {
            'Distance_km_typical': None,
            'Duree_hhmmss_typical': "00:00:00",
            'Distance_km_rush_hour': None,
            'Duree_hhmmss_rush_hour': "00:00:00",
            # 'Distance_km_current': None,
            # 'Duree_hhmmss_current': "00:00:00",
            'Success': False
        }

# === TEST AND MAIN ===
def test_audrey_colin():
    """Test specifically for Audrey Colin to understand the 43 vs 47 minute difference"""
    print("🧪 Testing Audrey Colin's commute...")
    
    global client
    client = googlemaps.Client(key=API_KEY)
    
    test_address = "128 Rue du Port, 34000 Frontignan"
    
    route_times = get_meaningful_route_times(
        client=client,
        origin_address=test_address,
        destination_address=company_address,
        mode="transit"
    )
    
    for time_type, data in route_times.items():
        if time_type != 'error':
            print(f"\n{time_type.upper()}: {data.get('duration')}")
            print(f"Description: {data.get('description')}")

def main():
    # Load data
    rh_path = "../data/DonneesRH.xlsx"
    df_rh = pd.read_excel(rh_path)
    
    global client
    client = googlemaps.Client(key=API_KEY)
    
    print(f"👥 Processing {len(df_rh)} employees...")
    print("🕐 Calculating: Typical conditions + Rush hour")
    print("-" * 60)
    
    results = []
    for idx, row in tqdm(df_rh.iterrows(), total=len(df_rh), desc="Calculating commutes"):
        commute_data = get_employee_commute_corrected(row, company_address)
        results.append(commute_data)
        time.sleep(random.uniform(2.0, 3.0))
    
    # Add to DataFrame
    for col in ['typical', 'rush_hour']:
        df_rh[f'Distance_km_{col}'] = [r[f'Distance_km_{col}'] for r in results]
        df_rh[f'Duree_hhmmss_{col}'] = [r[f'Duree_hhmmss_{col}'] for r in results]
    df_rh['Commute_Success'] = [r['Success'] for r in results]
    
    # Export
    output_file = "employee_commutes_two_times.csv"
    df_rh.to_csv(output_file, index=False)
    
    print(f"\n🎉 Done! File saved as: {output_file}")
    
    # Show Audrey Colin's results
    audrey = df_rh[(df_rh['Nom'] == 'Colin') & (df_rh['Prénom'] == 'Audrey')]
    if not audrey.empty:
        print(f"\n👤 Audrey Colin's results:")
        print(f"Typical: {audrey['Duree_hhmmss_typical'].iloc[0]}")
        print(f"Rush hour: {audrey['Duree_hhmmss_rush_hour'].iloc[0]}")
        #print(f"Current: {audrey['Duree_hhmmss_current'].iloc[0]}")

if __name__ == "__main__":
    # Test Audrey Colin first to see which time matches your original 43 minutes
    # test_audrey_colin()
    
    # Then run main processing
    main()

👥 Processing 161 employees...
🕐 Calculating: Typical conditions + Rush hour
------------------------------------------------------------


Calculating commutes:   0%|          | 0/161 [00:00<?, ?it/s]

📍 Calculating routes for Audrey Colin (transit)...
✅ Audrey Colin:
   Typical: 43 mins
   Rush hour: 47 mins


Calculating commutes:   1%|          | 1/161 [00:02<07:54,  2.97s/it]

📍 Calculating routes for Monique Ledoux (driving)...
✅ Monique Ledoux:
   Typical: 28 mins
   Rush hour: 37 mins


...

📍 Calculating routes for Odette Dumas (driving)...
✅ Odette Dumas:
   Typical: 18 mins
   Rush hour: 23 mins


Calculating commutes:  99%|█████████▉| 160/161 [07:47<00:02,  2.85s/it]

📍 Calculating routes for Henri Pineau (bicycling)...
✅ Henri Pineau:
   Typical: 21 mins
   Rush hour: 21 mins


Calculating commutes: 100%|██████████| 161/161 [07:50<00:00,  2.92s/it]



🎉 Done! File saved as: employee_commutes_two_times.csv

👤 Audrey Colin's results:
Typical: 00:42:33
Rush hour: 00:46:33


NameError: name 'df_rh' is not defined